In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cantera as ct
from CoolProp.CoolProp import PropsSI

# Pull values from Master Sheet

In [18]:
import os
import sys

# Navigate to the parent directory ('TheRocket')
os.chdir('C:/Users/Kidus/Desktop/TheRocket')
# Now import your module
import pythonExcelPull

C:\Users\Kidus\Desktop\TheRocket


In [20]:
import pandas as pd
import pythonExcelPull

#Must have openpyxl installed in python environment, separately from the pandas install! Pandas will call on this
# Read the Excel file
df = pd.read_excel('C:/Users/Kidus/Desktop/TheRocket/MasterParameters.xlsx')

# List of variables to pull from the Excel file
variables_to_pull = ['heightPayload',
                     'heightNoseCone',
                     'heightMainBay',
                     'heightAvBay',
                     'heightDrogueBay',
                     'heightCoupler3',
                     'heightOx',
                     'heightFuel',
                     'heightCoupler2',
                     'heightCoupler1',
                     'heightEngine',
                     'OD',
                     'tankThick'
]

# Creating an object to store excel variables
class Parameters:
    pass
ex = Parameters()

# Storing excel variables in object
for variable in variables_to_pull:
    value = pythonExcelPull.pythonExcelPull(df, variable)
    setattr(ex, variable, value)

# Input Chamber Parameters

In [22]:
To = 3200 # Stagnation temperature (Kelvin)
Po = 400 # Stagnation pressure (PSI)
OF = 10.2 # O/F ratio

# Input Chamber Geometry

In [23]:
# Pull from sizing notebook/spreadsheet

# Input Nitrous & Ethane States

In [ ]:
# Nitrous, saturated liquid at 750 PSI

rho_ni = 

# Density, Cp, Mu, Pr, Re, Nu, Temperature

# Ethane

# Density, Cp, Mu, Pr, Re, Nu, Temperature

# Input Cooling Channel Geometry

In [24]:
w = np.linspace(5e-4, 5e-3, 1000) # Channel width in meters
h = np.linspace(5e-4, 5e-3, 1000) # Channel hieght in meters

# Define Heat Analysis Functions

In [ ]:
def stat_temp(M, gamma, To):
    return To*(1 + ((gamma - 1)/2) * M**2)**(-1)

def stat_pressure(M, gamma, Po):
    return Po*(1 + ((gamma - 1)/2) * M**2)**(-(gamma)/(gamma - 1))

def T_rec():
    return stat_temp(M, gamma, To)*(1 + ((gamma - 1)/2)*((Pr)**(1/3))*(M**2))

def bartz(): # Bartz equation returns the enthalpy of the combusted gas
    return (0.026/(D*(0.2)))*((cp*((mu)**(0.2)))/(Pr**(0.6)))*((rho*v)**(0.8))*(rho_am/rho)*((mu_am/mu)**(0.2))

def cyl_conv_R():
    return np.log(r_o/r_i)/(2*np.pi*k)

def cyl_cond_R():
    return 1/(2*np.pi*r*h)

def R_net():
    
def heat_flow():
    return (T_h - T_c)/R_net

